In [ ]:
import holoviews as hv; hv.extension('bokeh', logo=False)
import numpy as np

In [ ]:
# Define domain
dx = 0.001
L = np.pi
x = L * np.arange(-1+dx,1+dx,dx)
n = len(x)
nquart = int(np.floor(n/4))

# Define hat function
f = np.zeros_like(x)
f[nquart:2*nquart] = (4/n)*np.arange(1,nquart+1)
f[2*nquart:3*nquart] = np.ones(nquart) - (4/n)*np.arange(0,nquart)

In [ ]:
# Compute Fourier series
A0  = np.sum(f * np.ones_like(x)) * dx
fFS = A0/2

A = np.zeros(20)
B = np.zeros(20)

curves = []
for k in range(20):
    A[k] = np.sum(f * np.cos(np.pi*(k+1)*x/L)) * dx # Inner product
    B[k] = np.sum(f * np.sin(np.pi*(k+1)*x/L)) * dx
    fFS = fFS + A[k]*np.cos((k+1)*np.pi*x/L) + B[k]*np.sin((k+1)*np.pi*x/L)
    curves.append( fFS )

hv.Overlay( [hv.Curve( (x,dat)) for dat in curves]).opts( hv.opts.Curve( width=500, show_grid=True))

In [ ]:
## Plot amplitudes

fFS  = (A0/2) * np.ones_like(f)
kmax = 100
A    = np.zeros(kmax)
B    = np.zeros(kmax)
ERR  = np.zeros(kmax)

A[0] = A0/2
ERR[0] = np.linalg.norm(f-fFS)/np.linalg.norm(f)

for k in range(1,kmax):
    A[k] = np.sum(f * np.cos(np.pi*k*x/L)) * dx
    B[k] = np.sum(f * np.sin(np.pi*k*x/L)) * dx
    fFS = fFS + A[k] * np.cos(k*np.pi*x/L) + B[k] * np.sin(k*np.pi*x/L)
    ERR[k] = np.linalg.norm(f-fFS)/np.linalg.norm(f)
    
thresh = np.median(ERR) * np.sqrt(kmax) * (4/np.sqrt(3))
r      = np.max(np.where(ERR > thresh))

def clamp(A,val):
    return [max(a,val) for a in A]

hv.Curve((np.arange(kmax),clamp(A, 1e-18)), "Error", "Coefficient" ).opts(width=500, logy=True, ylim=(1e-14,1.1), tools=['hover'], title="Fourier Coefficients")*\
hv.Scatter((r,A[r])).opts(size=8,color='red')+\
hv.Curve( (np.arange(kmax),clamp(ERR,1e-18)), "Index", "error" ).opts(logy=True)*\
hv.Scatter((r,ERR[r]), "Index", "Error").opts(size=8,color='red')